In [1]:
def Leiden(G, P):
    done = len(P) == len(G.nodes())
    while not done:
        P = MoveNodesFast(G, P)
        Prefined = RefinePartition(G, P)
        G = AggregateGraph(G, Prefined)
        P = [{v for v in C if v in G.nodes()} for C in Prefined]
        done = len(P) == len(G.nodes())
    return flat(P)

def MoveNodesFast(G, P):
    Q = list(G.nodes())
    while Q:
        v = Q.pop(0)
        C0 = max(P + [set()], key=lambda C: deltaHP(G, P, v, C))
        if deltaHP(G, P, v, C0) > 0:
            v_in_C = [v in C for C in P]
            P[v_in_C.index(True)].remove(v)
            P[v_in_C.index(True)].add(v)
            N = {u for u in G.neighbors(v) if u not in C0}
            Q.extend(list(N - set(Q)))
    return P

def RefinePartition(G, P):
    Prefined = SingletonPartition(G)
    for C in P:
        Prefined = MergeNodesSubset(G, Prefined, C)
    return Prefined

def MergeNodesSubset(G, P, S, gamma=0.5, theta=1):
    R = [v for v in S if E(G, v, S - {v}) >= gamma * k(G, v) * (len(S) - k(G, v))]
    np.random.shuffle(R)
    for v in R:
        if len({C for C in P if C.issubset(S) and E(G, C, S - C) >= gamma * len(C) * (len(S) - len(C))}) > 0:
            T = {C for C in P if C.issubset(S) and E(G, C, S - C) >= gamma * len(C) * (len(S) - len(C))}
            Pr = lambda C: np.exp(theta * deltaHP(G, P, v, C)) if deltaHP(G, P, v, C) >= 0 else 0
            Pr = np.array([Pr(C) for C in T])
            Pr /= np.sum(Pr)
            C0 = np.random.choice(list(T), p=Pr)
            v_in_C = [v in C for C in P]
            P[v_in_C.index(True)].remove(v)
            P[P.index(C0)].add(v)
        elif v not in flat(P):
            P.append({v})
    return P

def AggregateGraph(G, P):
    A = nx.Graph()
    for C in P:
        A.add_node(frozenset(C))
    for (u, v) in G.edges():
        for C in P:
            if u in C:
                Cu = frozenset(C)
            if v in C:
                Cv = frozenset(C)
        if A.has_edge(Cu, Cv):
            A[Cu][Cv]['weight'] += 1
        else:
            A.add_edge(Cu, Cv, weight=1)
    return A

def SingletonPartition(G):
    return [{v} for v in G.nodes()]

def flat(P):
    return {frozenset(C) for C in P}

def deltaHP(G, P, v, C):
    k_v = k(G, v

SyntaxError: incomplete input (931993161.py, line 65)